# Pre-Examination #2 - Give Me Some Credit

## Dataset Description:
### Dataset Kaggle Link:
[Kaggle Give Me Some Credit](https://www.kaggle.com/competitions/GiveMeSomeCredit/overview)

### Features:
| Feature Name                        | Description                                                                                                                                              | Type       |
|--------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------|------------|
| `SeriousDlqin2yrs`                     | Person experienced 90 days past due delinquency or worse                                                                                               | Y/N        |
| `RevolvingUtilizationOfUnsecuredLines` | Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits | percentage |
| `age`                                  | Age of borrower in years                                                                                                                                 | integer    |
| `NumberOfTime30-59DaysPastDueNotWorse` | Number of times borrower has been 30-59 days past due but no worse in the last 2 years.                                                                  | integer    |
| `DebtRatio`                            | Monthly debt payments, alimony,living costs divided by monthy gross income                                                                               | percentage |
| `MonthlyIncome`                        | Monthly income                                                                                                                                           | real       |
| `NumberOfOpenCreditLinesAndLoans`      | Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards)                                                     | integer    |
| `NumberOfTimes90DaysLate`              | Number of times borrower has been 90 days or more past due.                                                                                              | integer    |
| `NumberRealEstateLoansOrLines`         | Number of mortgage and real estate loans including home equity lines of credit                                                                           | integer    |
| `NumberOfTime60-89DaysPastDueNotWorse` | Number of times borrower has been 60-89 days past due but no worse in the last 2 years.                                                                  | integer    |
| `NumberOfDependents`                   | Number of dependents in family excluding themselves (spouse, children etc.)                                                                              | integer    |

### Target:
There is a Target Column in the dataset - `SeriousDlqin2yrs`, of datatype `boolean`, with 2 possible values - `Y/N` or, respectivelly, `1/0`. This column is showing if a person experienced 90 days past due delinquency or worse. Therefore, this problem is a Supervised Classification Machine Learning Problem.

### Problem Description:
Banks, in order to determine whether or not a loan should be granted to borrowers, require prior knowledge about borrower's capability to return the money they borrowed. For this, they use a system based on credibility that will offers a credit/reputation to borrowers. This credit is based on different criteria, such as: previous loans, overdebt, concurrent loans, and so on. This dataset is built upon the idea of prediction of probability that potential borrowers will experience financial distress in the next two years, enabling banks to decide better to grant a loan to that specific person or not. The task is to build a model that will predict this based on several features, like **Number of Days Overdue**, **Monthly Income** and others.

## Importing Prerequisites

In [ ]:
# Import Data Structures
import pandas as pd
import dask.dataframe as dd

# Import Data Manipulation Libraries
import numpy as np
import math

# Import Base Classes for Type Annotation
from sklearn.base import BaseEstimator, TransformerMixin
from numbers import Number

# Import Structure Manipulation Methods
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer, SimpleImputer
# from reparo import CDI, MICE

# Import Visualization Libs
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set_style(style="whitegrid")
sns.set_palette('bright')

# Import Outlier Detection
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

# Import Feature Selection Methods
from kydavra import PValueSelector
from sklearn.feature_selection import RFECV

# Import Hyperparameter Tuning
import optuna

# Import ML Models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Import Interpretation Metrics
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, roc_curve, average_precision_score, roc_auc_score, precision_recall_curve, PrecisionRecallDisplay

# from lime.lime_tabular import LimeTabularExplainer
# import shap

# Import Custom Utils
import swifter

## Dataset Loading
Since both training and test datasets are not very large, basic `pandas.DataFrame` will be sufficient.

In [ ]:
credit_train_zip: pd.DataFrame = pd.read_csv(filepath_or_buffer='dataset/cs-training.csv', sep=',')
credit_train_zip_eda: pd.DataFrame = credit_train_zip.copy()

In [ ]:
credit_test_zip: pd.DataFrame = pd.read_csv(filepath_or_buffer='dataset/cs-test.csv', sep=',')

### Basic Dataset Analysis

In [ ]:
credit_train_zip_eda.head(n=10)

In [ ]:
credit_train_zip_eda.tail(n=10)

As it may be seen, several features are present in the dataset. At the same time, several `NaN` values have been noticed in `MonthlyIncome` and `NumberOfDependents` columns. Besides that, no feature scaling was perfomed on this dataset. Also, first column of the dataset - `Unnamed: 0`, that is a result of the `pd.readcsv()` function, is a replacement for the first missing name in the original `.csv` file, that, most probably, is just the ID column for the samples in the dataset.

In [ ]:
credit_train_zip_eda.info()

There are no columns of `object` data type, which is often used for columns of `String` data type. Besides that there are 8 columns of `int64` data type and 4 columns of `float64` data type. Also, there are 2 columns with missing values - `MonthlyIncome` and `NumberOfDependents`, as it was mentioned in the previous paragraph. In total there are 11 Features and 1 Target Variable - `SeriousDlqin2yrs`. Again, the memory usage of this specific dataset is $\approx$ 13.7 MB.

In [ ]:
print(f"Train Dataset Shape: {credit_train_zip_eda.shape[0]} samples, {credit_train_zip_eda.shape[1]} columns")

As it may be seen, train dataset contains exactly 150000 training samples and 12 columns, as it was mentioned previously, 11 features and 1 target variable column.

In [ ]:
credit_train_zip_eda.describe()

After a brief analysis of the described dataset, were derived several conclusions:
1. Dataset is highly imbalanced, since target variable `SeriousDlqin2yrs` contains at least 75% of negative samples;
2. Features have different ranges of values, from very small range of continuous values - `age`, to very big ranges - `DebtRation` etc., which can impact gradient-based or distance-based Machine Learning Models, such as Logistic Regression or K-Nearest Neighbors;
3. There might be present a considerable amount of outliers, judging by the percentile values for several columns, such as: `ResolvingUtilizationOfUnsecuredLines` or `DebtRation`.

In [ ]:
credit_train_zip_eda.isnull().sum()

In [ ]:
missing_values_cols: list[str] = credit_train_zip_eda.isnull().sum()[credit_train_zip_eda.isnull().sum() > 0].index.tolist()
missing_values_cols

In [ ]:
for col in missing_values_cols:
    print(f"Feature '{col}': {credit_train_zip_eda[col].isnull().sum()} missing values ({(credit_train_zip_eda[col].isnull().sum() / credit_train_zip_eda.shape[0] * 100):.2f}%)")

There is a significant amount of missing values in `MonthlyIncome` column $\approx 19.82\%$, which is almost a fifth of the entire training set. In order to understand better how to treat those values, EDA should be performed to understand its distribution. On the other hand, column `NumberOfDependents` has a lower amount of missing values, $\approx 2.62\%$ which offers the possibility to drop those rows or use simple imputation mechanisms to treat them accordingly.

## Exploratory Data Analysis

In order to be able to explore and visualize data was used training set offered from Kaggle competition, in which null values were dropped in order to avoid problems at plotting the data.

In [ ]:
credit_train_zip_eda = credit_train_zip_eda.dropna()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))

sns.countplot(data=credit_train_zip_eda, x='SeriousDlqin2yrs', ax=axes[0])
axes[0].set_title('Distribution of Serious Delinquency in Training Set')
axes[0].set_xlabel('Serious Delinquency (1 = Yes, 0 = No)')
axes[0].set_ylabel('Count')

axes[1].pie(
    credit_train_zip_eda['SeriousDlqin2yrs'].value_counts(),
    autopct='%1.1f%%',
    labels=['No Serious Delinquency', 'Serious Delinquency']
)
axes[1].set_title('Pie Chart of Serious Delinquency in Training Set')

plt.tight_layout()
plt.show()

As it was mentioned previously and now proven, the dataset is highly imbalanced, therefore imbalance mitigation techniques should be applied. Further in the notebook will be presented several techniques to help with this issue.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.histplot(
    data=credit_train_zip_eda,
    x='age',
    bins=30,
    kde=True,
    ax=ax
)
ax.set_title('Age Distribution in Training Set')
ax.set_xlabel('Age')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    data=credit_train_zip_eda,
    x='age',
    ax=ax
)
ax.set_title('Age Distribution in Training Set')
ax.set_xlabel('Age')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

Column `age` is very close to the normal distribution, with several outlier points.

In [ ]:
def get_outliers_by_boxplot(df: pd.DataFrame, column: str) -> pd.Series:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_mask = (df[column] < lower_bound) | (df[column] > upper_bound)
    outliers_mask = outliers_mask.apply(
        lambda x: -1 if x else 1
    )

    return outliers_mask

In [ ]:
outliers_age_mask = get_outliers_by_boxplot(credit_train_zip_eda, 'age')
print(f"Total Number of Samples: {credit_train_zip_eda.shape[0]}")
print(f"Number of Outliers in Age Column detected using Boxplot Method: {outliers_age_mask[outliers_age_mask == -1].count()}")
print(f"Number of Inliers in Age Column detected using Boxplot Method: {outliers_age_mask[outliers_age_mask == 1].count()}")
print(f"Percentage of Outliers in Age Column detected using Boxplot Method: {(credit_train_zip_eda[outliers_age_mask == -1].shape[0] / credit_train_zip_eda.shape[0] * 100):.5f}%")
print(f"Percentage of Inliers in Age Column detected using Boxplot Method: {(credit_train_zip_eda[outliers_age_mask == 1].shape[0] / credit_train_zip_eda.shape[0] * 100):.5f}%")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    data=credit_train_zip_eda[outliers_age_mask == 1],
    x='age',
    ax=ax
)
ax.set_title('Age Distribution in Training Set')
ax.set_xlabel('Age')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

It looks better now, no outliers remained, but the effect of this modification should be analyzed further in the model training process.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    data=credit_train_zip_eda,
    x='MonthlyIncome',
    ax=ax
)
ax.set_title('Monthly Income Distribution in Training Set')
ax.set_xlabel('Monthly Income')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

As it may be noticed, the boxplot of `MonthlyIncome` displays a lot of outlier points. This may be a sign that Monthly Income is adhering to a Non-Normal Distribution, therefore should be treated different. However, this does not mean that this column does not contain any outliers at all or does not adhere to Normal Distribution, but a very skewed one.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.histplot(
    data=credit_train_zip_eda[credit_train_zip_eda['MonthlyIncome'] < 5e4],
    x='MonthlyIncome',
    bins=100,
    ax=ax
)

plt.tight_layout()
plt.show()

As it may be seen, a subset of the `MonthlyIncome` column was selected, since, judging by the boxplot, the most values are under $5\cdot10^4$ Monthly Income. This curve is very close to Gamma Curve. As a solution to bring this feature to a Normal Distribution is to apply Log Transformation. Thus, the data is brought to a Gaussian Distribution.

In [ ]:
credit_train_zip_eda['MonthlyIncome'] = np.log1p(credit_train_zip['MonthlyIncome'])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.histplot(
    data=credit_train_zip_eda,
    x='MonthlyIncome',
    bins=100,
    ax=ax
)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    data=credit_train_zip_eda,
    x='MonthlyIncome',
    ax=ax
)
ax.set_title('Monthly Income Distribution in Training Set')
ax.set_xlabel('Monthly Income')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

After Log Transformation, `MonthlyIncome` column got closer to Normal Distribution. However, it finds a large amount of outliers.

In [ ]:
outliers_income_mask = get_outliers_by_boxplot(credit_train_zip_eda, 'MonthlyIncome')
print(f"Total Number of Samples: {credit_train_zip_eda.shape[0]}")
print(f"Number of Outliers in Age Column detected using Boxplot Method: {outliers_income_mask[outliers_income_mask == -1].count()}")
print(f"Number of Inliers in Age Column detected using Boxplot Method: {outliers_income_mask[outliers_income_mask == 1].count()}")
print(f"Percentage of Outliers in Age Column detected using Boxplot Method: {(credit_train_zip_eda[outliers_income_mask == -1].shape[0] / credit_train_zip_eda.shape[0] * 100):.5f}%")
print(f"Percentage of Inliers in Age Column detected using Boxplot Method: {(credit_train_zip_eda[outliers_income_mask == 1].shape[0] / credit_train_zip_eda.shape[0] * 100):.5f}%")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    data=credit_train_zip_eda[outliers_income_mask == 1],
    x='MonthlyIncome',
    ax=ax
)
ax.set_title('Monthly Income Distribution in Training Set')
ax.set_xlabel('Monthly Income')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
outlier_detector = IsolationForest(
    contamination=0.05,
    random_state=42
)

In [ ]:
outlier_detector.fit(credit_train_zip_eda[['MonthlyIncome']])

In [ ]:
outliers_series = outlier_detector.predict(credit_train_zip_eda[['MonthlyIncome']])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    data=credit_train_zip_eda[outliers_series == 1],
    x='age',
    ax=ax
)
ax.set_title('Monthly Income Distribution in Training Set')
ax.set_xlabel('Monthly Income')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
print(f"Total Number of Samples: {credit_train_zip_eda.shape[0]}")
print(f"Number of Outliers in MonthlyIncome Column detected using Isolation Forest Method: {np.sum(outliers_series == -1)}")
print(f"Number of Inliers in MonthlyIncome Column detected using Isolation Forest Method: {np.sum(outliers_series == 1)}")
print(f"Percentage of Outliers in MonthlyIncome Column detected using Isolation Forest Method: {(credit_train_zip_eda[outliers_series == -1].shape[0] / credit_train_zip_eda.shape[0] * 100):.5f}%")
print(f"Percentage of Inliers in MonthlyIncome Column detected using Isolation Forest Method: {(credit_train_zip_eda[outliers_series == 1].shape[0] / credit_train_zip_eda.shape[0] * 100):.5f}%")

In both implementations of Outlier Remvoal, this feature looks better now, but since the dataset is highly imbalanced, $\approx 3.5%$ from Boxplot Method and $\approx 4.7%$ from Isolation Forest of the `MonthlyIncome` samples is quite big. A solution is to flag potential outliers and create a new feature that will comprise the far points of very low and very high `MonthlyIncome` instead of just dropping them, since they might be a result of natural variance from original scale of values and, respectivelly, have an impact on the target variable.

In [ ]:
min_age = credit_train_zip_eda['age'].min()
max_age = credit_train_zip_eda['age'].max()

age_bins = [min_age, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, max_age]
age_labels = [f'{min_age}-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', f'101-{max_age}']

age_binned = pd.cut(credit_train_zip_eda[outliers_income_mask == 1]['age'], bins=age_bins, labels=age_labels, include_lowest=True)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.barplot(
    x=age_binned,
    y=np.expm1(credit_train_zip_eda[outliers_income_mask == 1]['MonthlyIncome']),
    ax=axes[0]
)
axes[0].set_title('Monthly Income vs Age in Training Set')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Monthly Income')

sns.countplot(
    x=age_binned,
    ax=axes[1]
)

plt.tight_layout()
plt.show()

As it may be seen, highest amount of monthly income is achieved by people in between 41 and 70 years old. At the same time, the highest count of the people are in between the same range of age. Besides that, lowest income is presented by people from 21-30 years old and 90+ years old. However, since 90+ years old people count is very low, their income may be not very suggestive, due to the low variance in these specific age groups. Also, there is some anomaly in the dataset - there is or are multiple samples of people in between 0 and 10 years old with a high income. Since no error bar is displayed, there might be a single record within that age group, which, again, is not representative and is most probably noise. As a conclusion, middle age people are the persons that earn the highest amount of money monthly, while young adults and old people have on average lower income.

In [ ]:
credit_train_zip_eda['EstimatedMonthlyDebt'] = np.expm1(credit_train_zip_eda[outliers_income_mask == 1]['MonthlyIncome']) * credit_train_zip_eda[outliers_income_mask == 1]['DebtRatio']

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.barplot(
    x=age_binned,
    y=credit_train_zip_eda['EstimatedMonthlyDebt'],
    ax=axes[0]
)
axes[0].set_title('Estimated Monthly Debt (DebtRatio * MonthlyIncome) vs Age in Training Set')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Estimated Monthly Debt')

sns.countplot(
    x=age_binned,
    ax=axes[1]
)

plt.tight_layout()
plt.show()

Since `DebtRatio` represents borrowers monthly debt payments, living costs and alimony divided by their monthly gross income, from the reversed operation (`MonthlyIncome` $\cdot$ `DebtRatio`), may be estimated their monthly debt payment amount. As it may be noticed, similar as in previous case, the age group with highest amount of monthly debt are the people of middle age, between 41 and 70 years old, and people with lowest month debt are people of adult young age and old people. Therefore, there might be a direct proportioanl relationship between the monthly income and monthly debt, which is reasonable, since on average, people will borrow money if they have the finances to return them back.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.violinplot(
    x=credit_train_zip_eda[np.expm1(credit_train_zip_eda['MonthlyIncome']) < 5e4]['SeriousDlqin2yrs'],
    y=np.expm1(credit_train_zip_eda[np.expm1(credit_train_zip_eda['MonthlyIncome']) < 5e4]['MonthlyIncome']),
    hue=credit_train_zip_eda[np.expm1(credit_train_zip_eda['MonthlyIncome']) < 5e4]['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Serious Delinquency vs Monthly Income in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Monthly Income')

plt.tight_layout()
plt.show()

As it may be noticed, on the reduced training set, there is no significant difference in whether people encountered serious delinquency in the past 2 years, based on their monthly income. However, people with less income were slightly higher rate of encountering delinquency in the past 2 years than people with higher monthly income. Remembering the fact that this dataset is heavily imbalanced, this may not be very conclusive, since there are considerably less records about people that are in the possitive class of the target.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.histplot(
    data=credit_train_zip_eda,
    x='DebtRatio',
    bins=30,
    ax=ax
)

plt.tight_layout()
plt.show()

As it may be noticed, the histogram of the `DebtRatio` is inconclusive, due to the presence of outliers. In previous section was mentioned that `DebtRatio` at 75 percentile is present the value $\approx 0.87$, while maximum is $329664$, which obviously skews this feature. Therefore, this feature will cut to properly visualize it. Arbitrarily was selected value $2$ to exclude higher Debt Ratios from the dataset.

In [ ]:
(credit_train_zip_eda['DebtRatio'] > 2).value_counts()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    x=credit_train_zip_eda[credit_train_zip_eda['DebtRatio'] < 2]['SeriousDlqin2yrs'],
    y=credit_train_zip_eda[credit_train_zip_eda['DebtRatio'] < 2]['DebtRatio'],
    hue=credit_train_zip_eda[credit_train_zip_eda['DebtRatio'] < 2]['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Serious Delinquency vs Debt Ratio in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Debt Ratio')

plt.tight_layout()
plt.show()

As a result, people with higher Debt Ratio are slightly likely to experience 90 days past due delinquency, since the higher debt ratio means bigger difference between the total amount of monthly debt payments and their monthly income, making them less capable of properly paying back in time. People that were severily over due have an average Debt Ratio higher than $\approx 0.35$, while people with no serious delinquency - $\approx 0.27$.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.histplot(
    data=credit_train_zip_eda,
    x='RevolvingUtilizationOfUnsecuredLines',
    bins=100,
    ax=ax
)

plt.tight_layout()
plt.show()

In [ ]:
(credit_train_zip_eda['RevolvingUtilizationOfUnsecuredLines'] > 2).value_counts()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    x=credit_train_zip_eda[credit_train_zip_eda['RevolvingUtilizationOfUnsecuredLines'] < 2]['SeriousDlqin2yrs'],
    y=credit_train_zip_eda[credit_train_zip_eda['RevolvingUtilizationOfUnsecuredLines'] < 2]['RevolvingUtilizationOfUnsecuredLines'],
    hue=credit_train_zip_eda[credit_train_zip_eda['RevolvingUtilizationOfUnsecuredLines'] < 2]['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Serious Delinquency vs Revolving Utilization Of Unsecured Lines in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Revolving Utilization Of Unsecured Lines')

plt.tight_layout()
plt.show()

Revolving Utilization of Unsecured Lines in this dataset refers to the ratio between the total balance on credit cards and total credit limits. In other words, if borrower spents all the money that banks landed them, the Revolving Utilization will be equal to $1$, making them less trustworthy from the perspective of banks, since they uses all the money they borrowed, and it makes them less capable of paying back. As expected, persons with higher percentage are significantly inclined to experience delinquency.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=credit_train_zip_eda['NumberOfDependents'],
    ax=ax
)
ax.set_title('Distribution of Number of Dependents in Training Set')
ax.set_ylabel('NumberOfDependents')

plt.tight_layout()
plt.show()

Most borrowers have no dependents or at least 1 dependent.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=np.where(credit_train_zip_eda['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    hue=np.where(credit_train_zip_eda['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    ax=ax
)
ax.set_title('Distribution of Number of Dependents in Training Set')
ax.set_xlabel('NumberOfDependents')

plt.tight_layout()
plt.show()

In the above plot is represented the count plot of borrowers with no dependents, in other words - family members or people that rely on financial support of the borrower, and people with at least one dependent. All categories combined together are still slightly less than people with no dependents, but is now more comparable.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.barplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'],
    y=np.expm1(credit_train_zip_eda[outliers_income_mask == 1]['MonthlyIncome']),
    ax=axes[0]
)
axes[0].set_title('Monthly Income vs Number of Dependents in Training Set')
axes[0].set_xlabel('Number of Dependents')
axes[0].set_ylabel('Monthly Income')

sns.countplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'],
    ax=axes[1]
)

plt.tight_layout()
plt.show()

As it may be noticed, higher number of dependents results in higher monthly income, which is logic since more money is needed to support family and other dependents. However, there are small numbers of samples with borrowers with more than 3 dependents, therefore their monthly income is not precise and holds little variance.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.barplot(
    x=np.where(credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    y=np.expm1(credit_train_zip_eda[outliers_income_mask == 1]['MonthlyIncome']),
    hue=np.where(credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    ax=axes[0]
)
axes[0].set_title('Monthly Income vs Number of Dependents in Training Set')
axes[0].set_xlabel('Number of Dependents')
axes[0].set_ylabel('Monthly Income')

sns.countplot(
    x=np.where(credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    hue=np.where(credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    ax=axes[1]
)

plt.tight_layout()
plt.show()

As a result of aggregation, borrowers with no dependents, on average, have lower monthly income than people with at least one dependent.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.barplot(
    x=np.where(credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    y=np.expm1(credit_train_zip_eda[outliers_income_mask == 1]['MonthlyIncome']),
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[0]
)
axes[0].set_title('Monthly Income vs Number of Dependents in Training Set')
axes[0].set_xlabel('Number of Dependents')
axes[0].set_ylabel('Monthly Income')

sns.countplot(
    x=np.where(credit_train_zip_eda[outliers_income_mask == 1]['NumberOfDependents'] == 0, 'No Dependents', 'Has Dependents'),
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[1]
)

plt.tight_layout()
plt.show()

From the plot above may be derived that borrowers with at least one dependent and encountered delinquency has considerably lower monthly income that borrowers with one or more dependents and no delinquency. Similar trend is observed in the group of borrowers with no dependents at all.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    x=credit_train_zip_eda['SeriousDlqin2yrs'],
    y=credit_train_zip_eda['NumberOfOpenCreditLinesAndLoans'],
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Serious Delinquency vs Number of Open Credit Lines And Loans in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Number of Open Credit Lines And Loans')

plt.tight_layout()
plt.show()

No noticeable difference between the number of open credit lines and loans between borrowers that encountered serious delinquence and those who did not.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.lineplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberOfOpenCreditLinesAndLoans'],
    y=np.expm1(credit_train_zip_eda[outliers_income_mask == 1]['MonthlyIncome']) * credit_train_zip_eda[outliers_income_mask == 1]['DebtRatio'],
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[0]
)
axes[0].set_title('Estimated Monthly Debt (DebtRatio * MonthlyIncome) vs Number of Open Credit Lines And Loans in Training Set')
axes[0].set_xlabel('Number of Open Credit Lines And Loans')
axes[0].set_ylabel('Estimated Monthly Debt')

sns.countplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberOfOpenCreditLinesAndLoans'],
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[1]
)

plt.tight_layout()
plt.show()

As it may be observed, on average, borrowers have from 3 to 9 open credit lines and loans. Also, higher number of open loans results in higher estimated monthly debt. In the category of higher count of samples is distinguished slightly higher monthly debt for people that encountered serious delinquency.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.boxplot(
    x=credit_train_zip_eda['SeriousDlqin2yrs'],
    y=credit_train_zip_eda['NumberRealEstateLoansOrLines'],
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Serious Delinquency vs Number of Real Estate Loans Or Lines in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Number of Open Credit Lines And Loans')

plt.tight_layout()
plt.show()

Again, no significant difference in the distributions of Number of Real Estate Loans Or Lines between borrowers that manifeste delinquency and those who did not, which may be a result 

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.lineplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberRealEstateLoansOrLines'],
    y=credit_train_zip_eda['EstimatedMonthlyDebt'],
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[0]
)
axes[0].set_title('Estimated Monthly Debt (DebtRatio * MonthlyIncome) vs Number of Real Estate Loans Or Lines in Training Set')
axes[0].set_xlabel('Number of Open Credit Lines And Loans')
axes[0].set_ylabel('Estimated Monthly Debt')

sns.countplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberRealEstateLoansOrLines'],
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[1]
)

plt.tight_layout()
plt.show()

As it may be noticed, higher number of open credit lines and loans results in higher monthly debt. Since there are very few positive target samples, it may be inconclusive. However, there is no significant difference between the monthly debt based on the number of real estate loans.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.lineplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberRealEstateLoansOrLines'],
    y=np.expm1(credit_train_zip_eda[outliers_income_mask == 1]['MonthlyIncome']),
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[0]
)
axes[0].set_title('Monthly Income vs Number of Real Estate Loans Or Lines in Training Set')
axes[0].set_xlabel('Number of Open Credit Lines And Loans')
axes[0].set_ylabel('Monthly Income')

sns.countplot(
    x=credit_train_zip_eda[outliers_income_mask == 1]['NumberRealEstateLoansOrLines'],
    hue=credit_train_zip_eda[outliers_income_mask == 1]['SeriousDlqin2yrs'],
    ax=axes[1]
)

plt.tight_layout()
plt.show()

If plotted against the monthly income, there is a difference between borrowers with delinquency and without - those who manifeste serious delinquency had lower monthly income than those who did not manifested serious violations.

In [ ]:
credit_train_zip_eda['TotalNumberOfTimePastDue'] = credit_train_zip_eda['NumberOfTime30-59DaysPastDueNotWorse'] + credit_train_zip_eda['NumberOfTime60-89DaysPastDueNotWorse'] + credit_train_zip_eda['NumberOfTimes90DaysLate']

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.barplot(
    x=credit_train_zip_eda['SeriousDlqin2yrs'],
    y=credit_train_zip_eda['TotalNumberOfTimePastDue'],
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=axes[0]
)
axes[0].set_title('Total Number of Times Past Due vs Serious Delinquency in Training Set')
axes[0].set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
axes[0].set_ylabel('Total Number of Times Past Due')

sns.countplot(
    x=credit_train_zip_eda['TotalNumberOfTimePastDue'],
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=axes[1]
)

plt.tight_layout()
plt.show()

As it may be noticed, most people that encountered delinquency were past due with payback a lot more than borrowers that did not.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))

sns.lineplot(
    x=credit_train_zip_eda[(outliers_income_mask == 1) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 20)]['TotalNumberOfTimePastDue'],
    y=np.expm1(credit_train_zip_eda[(outliers_income_mask == 1) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 20)]['MonthlyIncome']),
    hue=credit_train_zip_eda[(outliers_income_mask == 1) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 20)]['SeriousDlqin2yrs'],
    ax=axes[0]
)
axes[0].set_title('Total Number of Times Past Due vs Serious Delinquency in Training Set')
axes[0].set_xlabel('Total Number of Times Past Due')
axes[0].set_ylabel('Monthly Income')

sns.countplot(
    x=credit_train_zip_eda[(outliers_income_mask == 1) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 20)]['TotalNumberOfTimePastDue'],
    hue=credit_train_zip_eda[(outliers_income_mask == 1) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 20)]['SeriousDlqin2yrs'],
    ax=axes[1]
)

plt.tight_layout()
plt.show()

Also, on the average, people that borrowed money from the banks and had a higher monthly income did not experience delinquency like people that had lower income and were at the same number of days past due with their payback.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.violinplot(
    x=credit_train_zip_eda[(credit_train_zip_eda['RevolvingUtilizationOfUnsecuredLines'] < 2) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 50)]['SeriousDlqin2yrs'],
    y=credit_train_zip_eda[(credit_train_zip_eda['RevolvingUtilizationOfUnsecuredLines'] < 2) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 50)]['TotalNumberOfTimePastDue'],
    hue=credit_train_zip_eda[(credit_train_zip_eda['RevolvingUtilizationOfUnsecuredLines'] < 2) & (credit_train_zip_eda['TotalNumberOfTimePastDue'] < 50)]['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Total Number of Times Past Due vs Serious Delinquency in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Total Number of Times Past Due')

plt.tight_layout()
plt.show()

Most people that were over due with their loans were more likely to manifest delinquency in the past 2 years, however, there are still people that manifested delinquency while having lower number of past due occurences, while people that did not encounter delinquency were mostly people with very low number of past due paybacks or even without them.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=credit_train_zip_eda['NumberOfTimes90DaysLate'],
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Distribution of Borrowers that had overdue 90+ days in Training Set')
ax.set_xlabel('Number of Times 90 Days Late')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Most records capture borrowers that did not pass the 90 days past due period, however several counts of at least 1 occurence of 90+ day overdue was recorded

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=np.where(credit_train_zip_eda['NumberOfTimes90DaysLate'] == 0, 'No 90+ days overdue', '90+ days overdue'),
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Distribution of Borrowers that had overdue 90+ days in Training Set')
ax.set_xlabel('Number of Times 90 Days Late')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

This feature is highly imbalanced. Taken in consideration that most people that did not encounter delinquency did not get past 90 days of not paying back, this number of people is greatly reduced in at least 1 occurence of that behavior, getting very close to the number of people that encountered delinquency and had at least one 90+ days overdue.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.violinplot(
    x=credit_train_zip_eda[credit_train_zip_eda['NumberOfTimes90DaysLate'] < 10]['SeriousDlqin2yrs'],
    y=credit_train_zip_eda[credit_train_zip_eda['NumberOfTimes90DaysLate'] < 10]['NumberOfTimes90DaysLate'],
    hue=credit_train_zip_eda[credit_train_zip_eda['NumberOfTimes90DaysLate'] < 10]['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Borrowers that had overdue 90+ days vs Serious Delinquency in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Total Number of Times Past Due')

plt.tight_layout()
plt.show()

Higher number of people that encountered delinquency had beed overdue 90+ days in comparison with the fewer number of people who did not manifest delinquency. 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=credit_train_zip_eda['NumberOfTime60-89DaysPastDueNotWorse'],
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Distribution of Borrowers that had overdue 60 to 89 days in Training Set')
ax.set_xlabel('Number of Times 60 to 89 Days Late Paybacks')
ax.set_ylabel('Count')

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=np.where(credit_train_zip_eda['NumberOfTime60-89DaysPastDueNotWorse'] == 0, 'No 60-89 days overdue', '60-89 days overdue'),
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Distribution of Borrowers that had overdure 60 to 89 days in Training Set')
ax.set_xlabel('Number of Times 60 to 89 Days Late')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Similar relationship as in previous case with 90+ days overdue paybacks.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.violinplot(
    x=credit_train_zip_eda[credit_train_zip_eda['NumberOfTime60-89DaysPastDueNotWorse'] < 10]['SeriousDlqin2yrs'],
    y=credit_train_zip_eda[credit_train_zip_eda['NumberOfTime60-89DaysPastDueNotWorse'] < 10]['NumberOfTime60-89DaysPastDueNotWorse'],
    hue=credit_train_zip_eda[credit_train_zip_eda['NumberOfTime60-89DaysPastDueNotWorse'] < 10]['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Borrowers that had overdue 60 to 89 days vs Serious Delinquency in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Total Number of Times Past Due')

plt.tight_layout()
plt.show()

Slightly higher number of people that encountered delinquency had beed overdue 60 to 89 days in comparison with the fewer number of people who did not manifest delinquency in the same range of past due occasions. 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=credit_train_zip_eda['NumberOfTime30-59DaysPastDueNotWorse'],
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Distribution of Borrowers that had overdure 30 to 59 days in Training Set')
ax.set_xlabel('Number of Times 30 to 59 Days Late Paybacks')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.countplot(
    x=np.where(credit_train_zip_eda['NumberOfTime30-59DaysPastDueNotWorse'] == 0, 'No 30-59 days overdue', '30-59 days overdue'),
    hue=credit_train_zip_eda['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Distribution of Borrowers that had overdure 30 to 59 days in Training Set')
ax.set_xlabel('Number of Times 30 to 59 Days Late')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Again, no significant difference than in previous cases of overdue. People that did not have overdue payments.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

sns.violinplot(
    x=credit_train_zip_eda[credit_train_zip_eda['NumberOfTime30-59DaysPastDueNotWorse'] < 10]['SeriousDlqin2yrs'],
    y=credit_train_zip_eda[credit_train_zip_eda['NumberOfTime30-59DaysPastDueNotWorse'] < 10]['NumberOfTime30-59DaysPastDueNotWorse'],
    hue=credit_train_zip_eda[credit_train_zip_eda['NumberOfTime30-59DaysPastDueNotWorse'] < 10]['SeriousDlqin2yrs'],
    ax=ax
)
ax.set_title('Borrowers that had overdue 30 to 59 days vs Serious Delinquency in Training Set')
ax.set_xlabel('Serious Delinquency (0 = No, 1 = Yes)')
ax.set_ylabel('Total Number of Times Past Due')

plt.tight_layout()
plt.show()

Significantly higher number of people that encountered delinquency had beed overdue 30 to 59 days in comparison with the fewer number of people who did not manifest delinquency in the same range of past due occasions. Also, many borrowers that had delinquency in the past 2 years had nmumerous past due paybacks, which decreases their credibility.

### Missing Values

In [ ]:
print(credit_train_zip.isnull().sum())

In [ ]:
print(credit_test_zip.isnull().sum())

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))

sns.countplot(x=credit_train_zip['MonthlyIncome'].isnull(), 
              ax=axes[0],
              hue=credit_train_zip['MonthlyIncome'].isnull()
              )
axes[0].set_title('Distribution of Monthly Income and Null Values in Training Set')
axes[0].set_xlabel('Is Monthly Income NaN')
axes[0].set_ylabel('Count')

axes[1].pie(
    credit_train_zip['MonthlyIncome'].isnull().value_counts(),
    autopct='%1.1f%%',
)
axes[1].set_title('Pie Chart of Monthly Income and Null Values in Training Set')

plt.tight_layout()
plt.show()

Since there is a significant portion of the dataset with missing monthly income values, more advanced imputation techniques should be used in order to replace them with numerical values. One way to do that is to select a subset of features that may affect the monthly income of the borrower and train a model-based Imputation techniques, such as: K-Nearest Neighbosr Imputer.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))

sns.countplot(x=credit_train_zip['NumberOfDependents'].isnull(), 
              ax=axes[0],
              hue=credit_train_zip['NumberOfDependents'].isnull()
              )
axes[0].set_title('Distribution of Number of Dependents and Null Values in Training Set')
axes[0].set_xlabel('Is Number of Dependents NaN')
axes[0].set_ylabel('Count')

axes[1].pie(
    credit_train_zip['NumberOfDependents'].isnull().value_counts(),
    autopct='%1.1f%%',
)
axes[1].set_title('Pie Chart of Number of Dependents and Null Values in Training Set')

plt.tight_layout()
plt.show()

There are not so many rows in `NumberOfDependents` column with missing values, therefore basic imputation technique may be used, such as: Median Imputation, since the distribution is skewed and there are no intermediate values, only integers.

## Model Selection

For the ML models that can be used, were analyzed the following options:
1. **Random Forest Classifier**, since it is robust to outliers and does not require scaling of the features to be done;
2. **XGBoost Classifier**, similar concept as Random Forest, but different implementation. It builds sequentially Decision Trees and each new tree corrects errors made by the previous ones. It is suitable when the dataset is large, as it is a fast algorithm, again - robust to outliers and feature scale;
3. **LightGBM Classifier**, similar to XGBoost, with slightly different tree building mechanism, faster training time, but less likely to find a generalized model and is less robust to outliers and noise in dataset.
3. **Logistic Regression** - the most basic classifier, fits the sigmoid function curve, offers fast computations and straight-forward interpretation of the results;
4. **Support Vector Machine Classifier** - powerful ML Model, but it requires data be standardized, which expects the data adhere to Gaussian Distribution. However, as it was described above, not all features are following this distribution. Also, it is very computationally expensive even on small datasets and does not support parallelization by default.
5. **K-Nearest Neighbors Classifier** - non-parametric (no assumption required for data distribution) algorithm with output being the class membership of a data point, based on the majority of type of its neighbors. Calculates the distance between each point, therefore is computationally intensive.

Since the dataset is not very large, will be attempted to train all of the above models, except Support Vector Machine Classifier, since it still is very computationally expensive, even on small dataset.

## Dataset Preparation

1. `_zip` - only `cs-train.csv` is used, for local score determination, models are trained on a subset of the Kaggle training set;
2. `_sub` - both `cs-train.csv` and `cs-test.csv` are used, for Kaggle submission of the predictions.

In [ ]:
credit_train_zip: pd.DataFrame = pd.read_csv(filepath_or_buffer='dataset/cs-training.csv', sep=',')
credit_test_zip: pd.DataFrame = pd.read_csv(filepath_or_buffer='dataset/cs-test.csv', sep=',')

In [ ]:
credit_train_zip_X: pd.DataFrame = credit_train_zip.drop(columns=['SeriousDlqin2yrs'])
credit_train_zip_y: pd.Series = credit_train_zip['SeriousDlqin2yrs']

In [ ]:
X_train_zip, X_test_zip, y_train_zip, y_test_zip = train_test_split(
    credit_train_zip_X,
    credit_train_zip_y,
    test_size=0.2,
    random_state=42
)

In [ ]:
X_train_sub: pd.DataFrame = credit_train_zip.drop(columns=['SeriousDlqin2yrs'])
y_train_sub: pd.Series = credit_train_zip['SeriousDlqin2yrs']
X_test_sub: pd.DataFrame = credit_test_zip.drop(columns=['SeriousDlqin2yrs'])

## Utils - General

In [ ]:
def plot_roc_curve_from_probabilities(y_true: np.ndarray, y_prob_pred: np.ndarray, estimator_name: str = None) -> None:
    fpr, tpr, _ = roc_curve(y_true=y_true, y_score=y_prob_pred, drop_intermediate=False)
    roc_auc = auc(fpr, tpr)
    display = RocCurveDisplay(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        estimator_name=estimator_name if estimator_name is not None else estimator_name.__class__.__name__
    )
    
    display.plot()
    plt.show()

In [ ]:
def plot_roc_curve_from_estimator(X: pd.DataFrame | np.ndarray, y: pd.Series | np.ndarray, estimator: BaseEstimator, estimator_name: str = None) -> None:
    RocCurveDisplay.from_estimator(
        estimator=estimator, 
        X=X, 
        y=y,
        name=estimator_name if estimator_name is not None else estimator.__class__.__name__
    )
    
    plt.show()

In [ ]:
def plot_precision_recall_curve_from_probabilities(y_true: np.ndarray, y_prob_pred: np.ndarray, pos_label: int = 1, estimator_name: str = None) -> None:
    precision, recall, _ = precision_recall_curve(y_true=y_true, y_score=y_prob_pred, pos_label=pos_label)
    display = RocCurveDisplay(
        precision=precision,
        recall=recall,
        estimator_name=estimator_name if estimator_name is not None else estimator_name.__class__.__name__
    )
    
    display.plot()
    plt.show()

In [ ]:
def plot_precision_recall_curve_from_estimator(X: pd.DataFrame | np.ndarray, y: pd.Series | np.ndarray, estimator: BaseEstimator, estimator_name: str = None) -> None:
    PrecisionRecallDisplay.from_estimator(
        estimator=estimator, 
        X=X,
        y=y,
        name=estimator_name if estimator_name is not None else estimator.__class__.__name__
    )
    
    plt.show()

In [ ]:
def print_classification_report(model_name: str,
                                y_true: pd.Series,
                                y_pred: pd.Series,
                                modifiers: str) -> None:
    print(f'{model_name} - {modifiers} - Accuracy Score: {accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(classification_report(y_true=y_true, y_pred=y_pred))

In [ ]:
def plot_confusion_matrix(model_name: str, 
                          y_true: pd.Series, 
                          y_pred: pd.Series,
                          modifiers: str) -> None:
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(f'{model_name} Classification Report - {modifiers}')
    plt.grid(False)
    plt.show()

In [ ]:
def save_predictions_to_sub_file(filepath_to_sample: str,
                                 filepath_to_save: str,
                                 y_pred: pd.Series,
                                 model_name: str,
                                 modifiers: str) -> None:
    sample_entry = pd.read_csv(filepath_to_sample)
    df_submission = pd.DataFrame({'Id': sample_entry['Id'], 'Probability': y_pred})

    df_submission.to_csv(f'{filepath_to_save}/submission_{modifiers}_{model_name}.csv', index=False)

In [ ]:
def create_pipeline(steps: list[tuple[str, BaseEstimator]]) -> Pipeline:
    return Pipeline(steps=steps).set_output(transform='pandas')

In [ ]:
model_performance_df: pd.DataFrame = pd.DataFrame(columns=['Score'])
model_performance_df.index = pd.MultiIndex.from_product(iterables=[[], []], names=['Model', 'Metric'])

In [ ]:
def compute_model_info_metrics(y_true: pd.Series | np.ndarray, y_pred: pd.Series | np.ndarray, y_pred_proba: pd.Series | np.ndarray, estimator_name: str) -> dict[str, Number]:
    model_specific_info_metrics: dict[str, dict | str] = {
        'Model_Name': estimator_name,
        'Model_Metrics': {
            'Accuracy': accuracy_score(y_true=y_true, y_pred=y_pred),
            'Precision (0)': precision_score(y_true=y_true, y_pred=y_pred, pos_label=0),
            'Recall (0)': recall_score(y_true=y_true, y_pred=y_pred, pos_label=0),
            'F1 Score (0)': f1_score(y_true=y_true, y_pred=y_pred, pos_label=0),
            'Precision (1)': precision_score(y_true=y_true, y_pred=y_pred, pos_label=1),
            'Recall (1)': recall_score(y_true=y_true, y_pred=y_pred, pos_label=1),
            'F1 Score (1)': f1_score(y_true=y_true, y_pred=y_pred, pos_label=1),
            'ROC AUC Score': roc_auc_score(y_true=y_true, y_score=y_pred_proba),
            'PC AUC Score': average_precision_score(y_true=y_true, y_score=y_pred_proba)
        }
    }

    return model_specific_info_metrics

In [ ]:
def add_to_df(metrics_dict: dict[str, Number], estimator_name: str, old_df: pd.DataFrame) -> pd.DataFrame:
    df_copy: pd.DataFrame = old_df.copy()

    index_iterables: list[str | list] = [[estimator_name], list(metrics_dict.keys())]
    new_multi_index = pd.MultiIndex.from_product(iterables=index_iterables, names=['Model', 'Metric'])

    new_scores = pd.DataFrame({'Score': list(metrics_dict.values())}, index=new_multi_index)

    df_copy = pd.concat([df_copy, new_scores])
    return df_copy

In [ ]:
def create_comparison_df(y_true: pd.Series | np.ndarray, 
                         y_pred: pd.Series | np.ndarray, 
                         y_pred_proba: pd.Series | np.ndarray) -> pd.DataFrame:
    comparison_df = pd.concat(
        [
            y_true.rename('y_true'),
            pd.Series(y_pred, index=y_true.index, name='y_pred'),
            pd.Series(y_pred_proba, index=y_true.index, name='y_pred_proba')
        ],
        axis=1
    )

    return comparison_df

In [ ]:
def plot_metrics(metrics_df: pd.DataFrame) -> None:
    metrics = metrics_df.index.get_level_values("Metric").unique()
    models = metrics_df.index.get_level_values("Model").unique()
    n_metrics = len(metrics)

    colors = ['red', 'orange', 'yellow', 'blue', 'black']
    color_map = dict(zip(models, colors))

    ncols = 2
    nrows = (n_metrics + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 3 * nrows), sharex=True)
    axes = axes.flatten()

    for idx, metric in enumerate(metrics):
        ax = axes[idx]
        ax.set_title(metric)

        for model in models:
            score = metrics_df.loc[(model, metric), "Score"]
            color = color_map[model]

            ax.hlines(
                y=score,
                xmin=0,
                xmax=1,
                label=model,
                color=color
            )
            
            ax.text(1.02, score, f"{score:.3f}", va='center', fontsize=8)
        
        ax.set_xlim(0, 1)
        ax.tick_params(left=True, labelleft=True)
        ax.grid(True)

    fig.suptitle("Model Comparison by Metric", fontsize=16)
    fig.supylabel("Score", fontsize=12)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', ncol=len(models), bbox_to_anchor=(0.5, 1.02))
    fig.legend(handles, labels, loc='lower center', ncol=len(models), bbox_to_anchor=(0.5, -0.02))

    plt.tight_layout()
    plt.show()

## Models Training - Baseline Performance

In order to observe the efficiency of dataset transformations and to compare model accuracy with the default model behavior on the unchanged dataset, with basic transformations that will ensure fit and predict workflows, training models with default parameters is done, which is known as baseline performance.

### Utils - Baseline Performance

In [ ]:
model_names: list[str] = ['K-Nearest Neighbors Classifier (BP)', 'LightGBM Classifier (BP)', 'Logistic Regression (BP)', 'Random Forest Classifier (BP)', 'XGBoost Classifier (BP)']

### Basic Transformations
Below are Transformers that are slightly changing the dataset in order to make model training possible, such as missing values imputation. At the same time, useless column that is responsible for ID is dropped.

In [ ]:
transformer_dropper_id = ColumnTransformer(
    transformers=[
        ('drop_id', 'drop', ['Unnamed: 0'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
transformer_log = FunctionTransformer(
    func=np.log1p,
    inverse_func=np.expm1,
    validate=False,
).set_output(transform='pandas')

In [ ]:
# mice_imputer: MICE = MICE()
simple_imputer_median: SimpleImputer = SimpleImputer(strategy='median').set_output(transform='pandas')
simple_imputer_mean: SimpleImputer = SimpleImputer(strategy='mean').set_output(transform='pandas')

In [ ]:
# class MICE_Wrapper(BaseEstimator, TransformerMixin):
#     def __init__(self, mice: MICE):
#         self.mice = mice
    
#     def fit(self, X, y = None):
#         return self
    
#     def transform(self, X):
#         X_copy = X.copy()
#         self.mice.apply(df=X_copy, columns=X_copy.columns)
#         return X_copy

#     def set_output(self, *, transform = None):
#         return self

In [ ]:
# mice_wrapper: MICE_Wrapper = MICE_Wrapper(mice=mice_imputer)

In [ ]:
null_transformer = ColumnTransformer(
    transformers=[
        ('impute_num_dependents', simple_imputer_median, ['NumberOfDependents']),
        # ('impute_mon_income', mice_wrapper),
        ('impute_mon_income', simple_imputer_mean, ['MonthlyIncome'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
pipeline_steps: list[tuple[str, TransformerMixin]] = [
    ('drop_id', transformer_dropper_id),
    ('impute_null', null_transformer),
    # ('log_transform', transformer_log),
    # ('impute_null_2', mice_wrapper)
]

In [ ]:
fit_transform_pipeline: Pipeline = create_pipeline(
    steps=pipeline_steps
)

In [ ]:
X_train_zip_tf = fit_transform_pipeline.fit_transform(
    X=X_train_zip,
    y=y_train_zip
)

In [ ]:
X_test_zip_tf = fit_transform_pipeline.transform(
    X=X_test_zip
)

In [ ]:
fit_transform_pipeline: Pipeline = create_pipeline(
    steps=pipeline_steps
)

In [ ]:
X_train_sub_tf = fit_transform_pipeline.fit_transform(
    X=X_train_sub,
    y=y_train_sub
)

In [ ]:
X_test_sub_tf = fit_transform_pipeline.transform(
    X=X_test_sub
)

### Light Gradient Boost Model (LightGBM)

In [ ]:
lightgbm_classifier = LGBMClassifier(n_jobs=-1, 
                                   random_state=42, 
                                   device='gpu', 
                                   verbose=2)

#### Local Scoring Training

In [ ]:
lightgbm_classifier.fit(X=X_train_zip_tf, y=y_train_zip)

In [ ]:
plot_roc_curve_from_estimator(X=X_test_zip_tf, y=y_test_zip, estimator=lightgbm_classifier, estimator_name='LightGBM Classifier')

In [ ]:
y_hat_zip_proba = lightgbm_classifier.predict_proba(X=X_test_zip_tf)[:, 1]

In [ ]:
roc_auc_score(y_true=y_test_zip, y_score=y_hat_zip_proba)

In [ ]:
plot_roc_curve_from_probabilities(y_true=y_test_zip, y_prob_pred=y_hat_zip_proba, estimator_name='LightGBM Classifier')

In [ ]:
y_hat_zip = lightgbm_classifier.predict(X=X_test_zip_tf)

In [ ]:
print_classification_report(model_name='LightGBM Classifier',
                            y_true=y_test_zip,
                            y_pred=y_hat_zip,
                            modifiers='Baseline Performance')

In [ ]:
plot_confusion_matrix(model_name='LightGBM',
                      y_true=y_test_zip,
                      y_pred=y_hat_zip,
                      modifiers='Baseline Performance')

As it may be seen, due to the fact that this dataset is highly imbalanced, accuracy is high, since model tends to predict only negative values, therefore precision, recall and F1 Score are very important, since they offer information about minority class separately.

In [ ]:
model_info_metrics: dict[str, dict | str] = compute_model_info_metrics(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba, estimator_name='LightGBM Classifier (BP)')

In [ ]:
model_performance_df = add_to_df(metrics_dict=model_info_metrics['Model_Metrics'], estimator_name=model_info_metrics['Model_Name'], old_df=model_performance_df)

In [ ]:
model_performance_df.xs('LightGBM Classifier (BP)')

In [ ]:
comparison_df = create_comparison_df(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba)

In [ ]:
comparison_df[comparison_df['y_true'] == 1]

In [ ]:
comparison_df[comparison_df['y_true'] == 0]

#### Kaggle Submission Training

In [ ]:
lightgbm_classifier = LGBMClassifier(n_jobs=-1, 
                                   random_state=42, 
                                   device='gpu', 
                                   verbose=2)

In [ ]:
lightgbm_classifier.fit(X=X_train_sub_tf, y=y_train_sub)

In [ ]:
y_hat_sub_proba = lightgbm_classifier.predict_proba(X=X_test_sub_tf)[:, 1]

In [ ]:
save_predictions_to_sub_file(filepath_to_sample='dataset/sampleEntry.csv',
                             filepath_to_save='dataset/submissions',
                             y_pred=y_hat_sub_proba,
                             model_name='lightgbm',
                             modifiers='default_bp')

Kaggle Score: `0.86675 | 0.86043`

### eXtreme Gradient Boosting (XGBoost)

In [ ]:
xgboost_classifier = XGBClassifier(n_jobs=-1, 
                                 random_state=42, 
                                 device='gpu', 
                                 verbosity=2)

#### Local Scoring Training

In [ ]:
xgboost_classifier.fit(X=X_train_zip_tf, y=y_train_zip)

In [ ]:
plot_roc_curve_from_estimator(X=X_test_zip_tf, y=y_test_zip, estimator=xgboost_classifier, estimator_name='XGBoost Classifier - BP')

In [ ]:
y_hat_zip_proba = xgboost_classifier.predict_proba(X=X_test_zip_tf)[:, 1]

In [ ]:
plot_roc_curve_from_probabilities(y_true=y_test_zip, y_prob_pred=y_hat_zip_proba, estimator_name='XGBoost Classifier - BP')

In [ ]:
y_hat_zip = xgboost_classifier.predict(X=X_test_zip_tf)

In [ ]:
print_classification_report(model_name='XGBoost Classifier',
                            y_true=y_test_zip,
                            y_pred=y_hat_zip,
                            modifiers='Baseline Performance')

In [ ]:
plot_confusion_matrix(model_name='XGBoost',
                      y_true=y_test_zip,
                      y_pred=y_hat_zip,
                      modifiers='Baseline Performance')

As it may be seen, due to the fact that this dataset is highly imbalanced, accuracy is high, since model tends to predict only negative values, therefore precision, recall and F1 Score are very important, since they offer information about minority class separately.

In [ ]:
model_info_metrics: dict[str, dict | str] = compute_model_info_metrics(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba, estimator_name='XGBoost Classifier (BP)')

In [ ]:
model_performance_df = add_to_df(metrics_dict=model_info_metrics['Model_Metrics'], estimator_name=model_info_metrics['Model_Name'], old_df=model_performance_df)

In [ ]:
model_performance_df.xs('XGBoost Classifier (BP)')

In [ ]:
comparison_df = create_comparison_df(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba)

In [ ]:
comparison_df[comparison_df['y_true'] == 1]

In [ ]:
comparison_df[comparison_df['y_true'] == 0]

#### Kaggle Submission Training

In [ ]:
xgboost_classifier = XGBClassifier(n_jobs=-1, 
                                   random_state=42, 
                                   device='gpu', 
                                   verbosity=2)

In [ ]:
xgboost_classifier.fit(X=X_train_sub_tf, y=y_train_sub)

In [ ]:
y_hat_sub_proba = xgboost_classifier.predict_proba(X=X_test_sub_tf)[:, 1]

In [ ]:
save_predictions_to_sub_file(filepath_to_sample='dataset/sampleEntry.csv',
                             filepath_to_save='dataset/submissions',
                             y_pred=y_hat_sub_proba,
                             model_name='xgboost',
                             modifiers='default_bp')

Kaggle Score: `0.85811 | 0.85248`

### Logistic Regression

In [ ]:
logistic_regressor = LogisticRegression(n_jobs=-1, 
                                        verbose=2)

#### Local Scoring Training

In [ ]:
logistic_regressor.fit(X=X_train_zip_tf, y=y_train_zip)

In [ ]:
plot_roc_curve_from_estimator(X=X_test_zip_tf, y=y_test_zip, estimator=logistic_regressor, estimator_name='Logistic Regression (BP)')

In [ ]:
y_hat_zip_proba = logistic_regressor.predict_proba(X=X_test_zip_tf)[:, 1]

In [ ]:
plot_roc_curve_from_probabilities(y_true=y_test_zip, y_prob_pred=y_hat_zip_proba, estimator_name='Logistic Regression (BP)')

In [ ]:
y_hat_zip = logistic_regressor.predict(X=X_test_zip_tf)

In [ ]:
print_classification_report(model_name='Logistic Regressor',
                            y_true=y_test_zip,
                            y_pred=y_hat_zip,
                            modifiers='Baseline Performance')

In [ ]:
plot_confusion_matrix(model_name='Logistic Regressor',
                      y_true=y_test_zip,
                      y_pred=y_hat_zip,
                      modifiers='Baseline Performance')

As it may be seen, due to the fact that this dataset is highly imbalanced, accuracy is high, since model tends to predict only negative values, therefore precision, recall and F1 Score are very important, since they offer information about minority class separately.

In [ ]:
model_info_metrics: dict[str, dict | str] = compute_model_info_metrics(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba, estimator_name='Logistic Regression (BP)')

In [ ]:
model_performance_df = add_to_df(metrics_dict=model_info_metrics['Model_Metrics'], estimator_name=model_info_metrics['Model_Name'], old_df=model_performance_df)

In [ ]:
model_performance_df.xs('Logistic Regression (BP)')

In [ ]:
comparison_df = create_comparison_df(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba)

In [ ]:
comparison_df[comparison_df['y_true'] == 1]

In [ ]:
comparison_df[comparison_df['y_true'] == 0]

#### Kaggle Submission Training

In [ ]:
logistic_regressor = LogisticRegression(n_jobs=-1, 
                                        verbose=2)

In [ ]:
logistic_regressor.fit(X=X_train_sub_tf, y=y_train_sub)

In [ ]:
y_hat_sub = logistic_regressor.predict_proba(X=X_test_sub_tf)[:, 1]

In [ ]:
save_predictions_to_sub_file(filepath_to_sample='dataset/sampleEntry.csv',
                             filepath_to_save='dataset/submissions',
                             y_pred=y_hat_sub,
                             model_name='logistic',
                             modifiers='default_bp')

Kaggle Score: `0.83524 | 0.82670`

### Random Forest Classifier

In [ ]:
random_forest_classifier = RandomForestClassifier(n_jobs=-1, 
                                                  random_state=42, 
                                                  verbose=2)

#### Local Scoring Training

In [ ]:
random_forest_classifier.fit(X=X_train_zip_tf, y=y_train_zip)

In [ ]:
plot_roc_curve_from_estimator(X=X_test_zip_tf, y=y_test_zip, estimator=random_forest_classifier, estimator_name='Random Forest Classifier - BP')

In [ ]:
y_hat_zip_proba = random_forest_classifier.predict_proba(X=X_test_zip_tf)[:, 1]

In [ ]:
plot_roc_curve_from_probabilities(y_true=y_test_zip, y_prob_pred=y_hat_zip_proba, estimator_name='Random Forest Classifier - BP')

In [ ]:
y_hat_zip = random_forest_classifier.predict(X=X_test_zip_tf)

In [ ]:
print_classification_report(model_name='Random Forest Classifier',
                            y_true=y_test_zip,
                            y_pred=y_hat_zip,
                            modifiers='Baseline Performance')

In [ ]:
plot_confusion_matrix(model_name='Random Forest',
                      y_true=y_test_zip,
                      y_pred=y_hat_zip,
                      modifiers='Baseline Performance')

In [ ]:
model_info_metrics: dict[str, dict | str] = compute_model_info_metrics(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba, estimator_name='Random Forest Classifier (BP)')

In [ ]:
model_performance_df = add_to_df(metrics_dict=model_info_metrics['Model_Metrics'], estimator_name=model_info_metrics['Model_Name'], old_df=model_performance_df)

In [ ]:
model_performance_df.xs('Random Forest Classifier (BP)')

In [ ]:
comparison_df = create_comparison_df(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba)

In [ ]:
comparison_df[comparison_df['y_true'] == 1]

In [ ]:
comparison_df[comparison_df['y_true'] == 0]

#### Kaggle Submission Training

In [ ]:
random_forest_classifier = RandomForestClassifier(n_jobs=-1, 
                                                  random_state=42, 
                                                  verbose=2)

In [ ]:
random_forest_classifier.fit(X=X_train_sub_tf, y=y_train_sub)

In [ ]:
y_hat_sub = random_forest_classifier.predict_proba(X=X_test_sub_tf)[:, 1]

In [ ]:
save_predictions_to_sub_file(filepath_to_sample='dataset/sampleEntry.csv',
                             filepath_to_save='dataset/submissions',
                             y_pred=y_hat_sub,
                             model_name='randomForest',
                             modifiers='default_bp')

Kaggle Score: `0.84255 | 0.83637`

### K-Nearest Neighbors Classifier

In [ ]:
k_nearest_neighbors_classifier = KNeighborsClassifier(n_jobs=-1)

#### Local Scoring Training

In [ ]:
k_nearest_neighbors_classifier.fit(X=X_train_zip_tf, y=y_train_zip)

In [ ]:
plot_roc_curve_from_estimator(X=X_test_zip_tf, y=y_test_zip, estimator=k_nearest_neighbors_classifier, estimator_name='K-Nearest Neighbors Classifier - BP')

In [ ]:
y_hat_zip_proba = k_nearest_neighbors_classifier.predict_proba(X=X_test_zip_tf)[:, 1]

In [ ]:
plot_roc_curve_from_probabilities(y_true=y_test_zip, y_prob_pred=y_hat_zip_proba, estimator_name='K-Nearest Neighbors Classifier - BP')

In [ ]:
y_hat_zip = k_nearest_neighbors_classifier.predict(X=X_test_zip_tf)

In [ ]:
print_classification_report(model_name='K-Nearest Neighbors Classifier',
                            y_true=y_test_zip,
                            y_pred=y_hat_zip,
                            modifiers='Baseline Performance')

In [ ]:
plot_confusion_matrix(model_name='K-Nearest Neighbors',
                      y_true=y_test_zip,
                      y_pred=y_hat_zip,
                      modifiers='Baseline Performance')

In [ ]:
model_info_metrics: dict[str, dict | str] = compute_model_info_metrics(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba, estimator_name='K-Nearest Neighbors Classifier (BP)')

In [ ]:
model_performance_df = add_to_df(metrics_dict=model_info_metrics['Model_Metrics'], estimator_name=model_info_metrics['Model_Name'], old_df=model_performance_df)

In [ ]:
model_performance_df.xs('K-Nearest Neighbors Classifier (BP)')

In [ ]:
comparison_df = create_comparison_df(y_true=y_test_zip, y_pred=y_hat_zip, y_pred_proba=y_hat_zip_proba)

In [ ]:
comparison_df[comparison_df['y_true'] == 1]

In [ ]:
comparison_df[comparison_df['y_true'] == 0]

#### Kaggle Submission Training

In [ ]:
k_nearest_neighbors_classifier = KNeighborsClassifier(n_jobs=-1)

In [ ]:
k_nearest_neighbors_classifier.fit(X=X_train_sub_tf, y=y_train_sub)

In [ ]:
y_hat_sub = k_nearest_neighbors_classifier.predict_proba(X=X_test_sub_tf)[:, 1]

In [ ]:
save_predictions_to_sub_file(filepath_to_sample='dataset/sampleEntry.csv',
                             filepath_to_save='dataset/submissions',
                             y_pred=y_hat_sub,
                             model_name='knn',
                             modifiers='default_bp')

Kaggle Score: `0.74397 | 0.73065`

### Baseline Performance Summary
Since dataset is highly imbalanced, all models tend to predict probability values close to `0` for a vast majority of test rows, that are also mostly close to `0`, therefore their accuracy on Kaggle website is pretty similar, fluctuating around values of $\approx 0.74$ achieved by K-Nearest Neighbors Classifier to $\approx 0.86$ achieved by LightGBM Classifier, thus an important step is class balancing, in order to provide better scores of Precision, Recall and ROC-AUC, that are very useful in determination of real behavior of the model and its predictions.

#### Kaggle Results Table:
| **Classifier Name**   | **Modifiers** | **Private Score** | **Public Score** |
|-----------------------|---------------|-------------------|------------------|
| _LightGBM_            |       BP      |      0.86043      |      0.86043     |
| _XGBoost_             |       BP      |      0.85911      |      0.85248     |
| _Logistic Regression_ |       BP      |      0.83524      |      0.82670     |
| _Random Forest_       |       BP      |      0.84255      |      0.83637     |
| _K-Nearest Neighbors_ |       BP      |      0.74397      |      0.73065     |

#### Local Perfomance Metrics

In [ ]:
model_performance_df.loc[model_names]

In [ ]:
plot_metrics(metrics_df=model_performance_df.loc[model_names])

Was observed that all models suffer from misclassification of test samples with positive label, as a result of highly imbalanced dataset ($\approx 93\%$ negative labels against $\approx 7\%$ positive labels). As a mitigation strategy for this is to implement Class Balancing methods, either Undersampling, which will reduce the majority class to match closely the minority class, or Oversampling, that will create new samples of minority class to reduce the proportion of majority class, use `scale_pos_weight` or `class_weight` hyperparameters for trained models, which will associate weights to classes based on their class frequencies in the dataset. Since ROC-AUC Score is high for boosted trees algorithms

In [ ]:
# TODO: scale pos weight xgboost lightgbm ratio negative/positive classes, trees handle automatic Null values, FE + FSc + OUTLIERS + CLASS BALANCING + FSel + etc.